In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip,VerticalFlip,Rotate

In [2]:
""" Create a directory """

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def load_data(path):
    
    train_x = sorted(glob(os.path.join(path,"training","images","*tif")))
    train_y = sorted(glob(os.path.join(path,"training","1st_manual","*gif")))
    
    
    test_x = sorted(glob(os.path.join(path,"test","images","*tif")))
    test_y = sorted(glob(os.path.join(path,"test","1st_manual","*gif")))
    
    return (train_x,train_y),(test_x,test_y)
    

In [4]:
def augment_data(images, masks, save_path, augment=True):
    size = (512, 512)

    for idx, (x, y) in tqdm(enumerate (zip(images, masks)), total=len(images)):
        """ Extracting the name """
        name = x.split("/")[-1].split(".")[0]
        
        """ Reading image and mask """
        
        x = cv2.imread(x,cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0]
        
        if augment == True:
            
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x,mask=y)
            
            x1 = augmented["image"]
            y1 = augmented["mask"]
            
            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x,mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]
            
            aug = Rotate(limit=45,p=1.0)
            augmented = aug(image=x,mask=y)
            x3 = augmented["image"]
            y3= augmented["mask"]
            
            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]
        
        else :
            X = [x]
            Y = [y]
            
            
        index = 0    
        for i,m in zip(X,Y):
            
            i = cv2.resize(i,size)
            m = cv2.resize(m,size)
            
            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path,"image",tmp_image_name)
            mask_path = os.path.join(save_path,"mask",tmp_mask_name)
            
            cv2.imwrite(image_path,i)
            cv2.imwrite(mask_path,m)
            
            index += 1
                   

In [5]:
if __name__ == "__main__":
    
    """ Seeding """
    np.random.seed(42)
    
    """ Load the data """
    data_path= "/kaggle/input/drive2004/DRIVE"
    (train_x,train_y),(test_x,test_y) = load_data(data_path)

In [6]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")



""" Create directories to save the augmented data """

create_dir("new_data/train/image/")
create_dir("new_data/train/mask/")
create_dir("new_data/test/image/")
create_dir("new_data/test/mask/")


Train: 20 - 20
Test: 20 - 20


In [7]:
augment_data(train_x,train_y,"new_data/train/",augment=True)
augment_data(test_x,test_y,"new_data/test/",augment=False)

100%|██████████| 20/20 [00:01<00:00, 10.39it/s]


In [8]:
import torch
import torch.nn as nn

In [9]:
import torch
import torch.nn as nn


class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        return x
    

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))
        
    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        
        return x, p
    

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c + out_c, out_c)
        
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        
        return x
    

class build_unet(nn.Module):
    def __init__(self):
        super().__init__()
        
        """ Encoder """
        
        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)
        
        """ Bottleneck """
        
        self.b = conv_block(512, 1024)
        
        """ Decoder """
        
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)
        
        """ Classifier """
        self.outputs = nn.Conv2d(64,1,kernel_size=1,padding=0)
        
    def forward(self, inputs):
        """Encoder"""
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)
        
        """ Bottleneck """
        
        b = self.b(p4)
        
        
        
        """ Decoder """
        
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)
        
        outputs = self.outputs(d4)
        
        return outputs  
        

if __name__ == "__main__":
    x = torch.randn((2, 3, 512, 512))
    f = build_unet()
    y = f(x)
    print(y.shape)

torch.Size([2, 1, 512, 512])


In [32]:
import random
import numpy as np
import torch


def train(model,loader,optimizer,loss_fn,device):
    epoch_loss = 0.0 
    model.train()
    for x,y in loader:
        x = x.to(device,dtype=torch.float32)
        y = y.to(device,dtype=torch.float32)
        
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    epoch_loss = epoch_loss/len(loader)
    return epoch_loss


def evaluate(model,loader,loss_fn,device):
    
    epoch_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device,dtype=torch.float32)
            y = y.to(device,dtype=torch.float32)

            
            y_pred = model(x)
            loss = loss_fn(y_pred,y)
            epoch_loss += loss.item()
        
        epoch_loss = epoch_loss/len(loader)
        
    return epoch_loss
            
            
def seeding(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Directories """
    create_dir("files")

    """ Load dataset"""

    train_x = sorted(glob("/kaggle/working/new_data/train/image/*"))[:20]
    train_y = sorted(glob("/kaggle/working/new_data/train/mask/*"))[:20]

    valid_x = sorted(glob("/kaggle/working/new_data/test/image/*"))
    valid_y = sorted(glob("/kaggle/working/new_data/test/mask/*")) 

    data_str = f"Dataset Size:\nTrain:{len(train_x)}-Valid:{len(valid_x)}\n"
    print(data_str)


Dataset Size:
Train:20-Valid:20



In [11]:
H = 512
W = 512

size = (H,W)
batch_size = 2
num_epochs = 50
lr = 1e-4
checkpoint_path = "files/checkpoint.pth"

In [24]:
""" Dataset and the loader """
from torch.utils.data import Dataset


class DriveDataset(Dataset):
    
    def __init__(self, images_path, masks_path):
        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)
        
    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image / 255.0 
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        
        """ Reading masks """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask / 255.0 
        mask = np.expand_dims(mask, axis=0)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)
        
        return image, mask
    
    def __len__(self):
        return self.n_samples


In [25]:
train_dataset = DriveDataset(train_x,train_y)
valid_dataset = DriveDataset(valid_x,valid_y)

In [26]:
""" Loader """
from torch.utils.data import DataLoader
train_loader = DataLoader(
    
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 2
)

valid_loader = DataLoader(
    
    dataset = valid_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 2
)


In [27]:
device = torch.device('cuda')
model = build_unet()
model = model.to(device)

In [28]:
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

In [29]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience=5,verbose=True)
loss_fn = DiceBCELoss()

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
""" Training the model """
import time

best_valid_loss = float("inf")

for epoch in range(num_epochs):
    start_time = time.time()
    
    
    train_loss = train(model,train_loader,optimizer,loss_fn,device)
    valid_loss = evaluate(model,valid_loader,loss_fn,device)
    
    """ Saving the model """
    
    if valid_loss<best_valid_loss:
        data_str = f"Valid loss imporved from {best_valid_loss:2.4f} to {valid_loss}"
        print(data_str)
    
    end_time = time.time()
    
    epoch_mins,epoch_secs = epoch_time(start_time,end_time)
    
    data_str = f'Epoch : {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s'
    data_str += f'\tTrain Loss: {train_loss: .3f}\n'
    data_str += f'\t Val.Loss : {valid_loss: .3f}\n'
    print(data_str)
    

Valid loss imporved from inf to 0.49821129739284514
Epoch : 01 | Epoch Time: 0m 3s	Train Loss:  0.290
	 Val.Loss :  0.498

Valid loss imporved from inf to 0.5099482417106629
Epoch : 02 | Epoch Time: 0m 3s	Train Loss:  0.289
	 Val.Loss :  0.510

Valid loss imporved from inf to 0.4942701131105423
Epoch : 03 | Epoch Time: 0m 3s	Train Loss:  0.286
	 Val.Loss :  0.494

Valid loss imporved from inf to 0.49403526484966276
Epoch : 04 | Epoch Time: 0m 3s	Train Loss:  0.285
	 Val.Loss :  0.494

Valid loss imporved from inf to 0.4915975838899612
Epoch : 05 | Epoch Time: 0m 4s	Train Loss:  0.278
	 Val.Loss :  0.492

Valid loss imporved from inf to 0.48781625032424925
Epoch : 06 | Epoch Time: 0m 3s	Train Loss:  0.275
	 Val.Loss :  0.488

Valid loss imporved from inf to 0.48522281348705293
Epoch : 07 | Epoch Time: 0m 3s	Train Loss:  0.273
	 Val.Loss :  0.485

Valid loss imporved from inf to 0.48113332986831664
Epoch : 08 | Epoch Time: 0m 3s	Train Loss:  0.270
	 Val.Loss :  0.481

Valid loss imporved